# Khám Phá và Tiền Xử Lý Dữ Liệu Cầu Thủ
## Dự án: Dự đoán Giá Cầu Thủ Bóng Đá

Notebook này thực hiện các bước:
1. Đọc và khám phá dữ liệu (EDA)
2. Phân tích phân phối và tương quan
3. Tiền xử lý dữ liệu
4. Lưu dữ liệu đã xử lý

In [6]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
pd.set_option('display.max_columns', None)

print("✅ Đã import các thư viện thành công!")

ModuleNotFoundError: No module named 'matplotlib'

## 1. Đọc Dữ Liệu

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('../data/raw/sofifa_players.csv')

print(f"📊 Kích thước dữ liệu: {df.shape}")
print(f"📊 Số cầu thủ: {df.shape[0]:,}")
print(f"📊 Số thuộc tính: {df.shape[1]}")
print("\n5 dòng đầu tiên:")
df.head()

In [ ]:
# Thông tin tổng quan
print("📋 Thông tin các cột:")
df.info()

## 2. Phân Tích Dữ Liệu Khám Phá (EDA)

In [ ]:
# Kiểm tra giá trị thiếu
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

print(f"❌ Tổng số giá trị thiếu: {df.isnull().sum().sum():,}")
print(f"\n📊 Top 10 cột có nhiều giá trị thiếu nhất:")
print(missing_data.head(10))

# Visualize
plt.figure(figsize=(12, 6))
missing_data.head(15).plot(kind='bar', color='coral')
plt.title('Top 15 Cột Có Nhiều Giá Trị Thiếu Nhất', fontsize=14, fontweight='bold')
plt.xlabel('Tên Cột', fontsize=12)
plt.ylabel('Số Giá Trị Thiếu', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Thống kê mô tả các biến số
print("📊 Thống kê mô tả:")
df.describe()

In [3]:
# Phân tích biến target (Value_Numeric)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df['Value_Numeric'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_title('Phân Phối Giá Trị Cầu Thủ', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Giá Trị (€)', fontsize=12)
axes[0].set_ylabel('Tần Số', fontsize=12)
axes[0].axvline(df['Value_Numeric'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df['Value_Numeric'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

# Boxplot
axes[1].boxplot(df['Value_Numeric'], vert=True)
axes[1].set_title('Boxplot Giá Trị Cầu Thủ', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Giá Trị (€)', fontsize=12)

# Log transformation
axes[2].hist(np.log1p(df['Value_Numeric']), bins=50, color='lightgreen', edgecolor='black')
axes[2].set_title('Phân Phối Log(Giá Trị)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Log(Giá Trị)', fontsize=12)
axes[2].set_ylabel('Tần Số', fontsize=12)

plt.tight_layout()
plt.show()

print(f"💰 Giá trị trung bình: €{df['Value_Numeric'].mean():,.0f}")
print(f"💰 Giá trị trung vị: €{df['Value_Numeric'].median():,.0f}")
print(f"💰 Giá trị cao nhất: €{df['Value_Numeric'].max():,.0f}")
print(f"💰 Giá trị thấp nhất: €{df['Value_Numeric'].min():,.0f}")

NameError: name 'plt' is not defined

In [ ]:
# Phân tích tương quan với biến target
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
correlation = df[numerical_cols].corr()['Value_Numeric'].sort_values(ascending=False)

print("🔗 Top 15 thuộc tính có tương quan mạnh nhất với Giá Trị:")
print(correlation.head(15))

# Visualize
plt.figure(figsize=(10, 8))
correlation.head(15).plot(kind='barh', color='teal')
plt.title('Top 15 Tương Quan với Giá Trị Cầu Thủ', fontsize=14, fontweight='bold')
plt.xlabel('Hệ Số Tương Quan', fontsize=12)
plt.ylabel('Thuộc Tính', fontsize=12)
plt.tight_layout()
plt.show()

In [5]:
# Ma trận tương quan cho các thuộc tính quan trọng
important_features = ['Value_Numeric', 'Overall', 'Potential', 'Age', 'Wage_Numeric',
                      'Reactions', 'Ball_control', 'Short_passing', 'Dribbling', 'Sprint_speed']
important_features = [col for col in important_features if col in df.columns]

plt.figure(figsize=(12, 10))
sns.heatmap(df[important_features].corr(), annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, fmt='.2f', cbar_kws={"shrink": 0.8})
plt.title('Ma Trận Tương Quan - Các Thuộc Tính Quan Trọng', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

NameError: name 'df' is not defined

In [4]:
# Phân tích phân loại
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Preferred Foot
df['Preferred_Foot'].value_counts().plot(kind='bar', ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Phân Bố Chân Thuận', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Chân Thuận')
axes[0,0].set_ylabel('Số Lượng')

# Age Distribution
axes[0,1].hist(df['Age'], bins=30, color='lightcoral', edgecolor='black')
axes[0,1].set_title('Phân Bố Tuổi Cầu Thủ', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Tuổi')
axes[0,1].set_ylabel('Tần Số')

# Overall Rating
axes[1,0].hist(df['Overall'], bins=30, color='lightgreen', edgecolor='black')
axes[1,0].set_title('Phân Bố Chỉ Số Overall', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Overall Rating')
axes[1,0].set_ylabel('Tần Số')

# Nationality - Top 10
df['Nationality'].value_counts().head(10).plot(kind='barh', ax=axes[1,1], color='gold')
axes[1,1].set_title('Top 10 Quốc Gia Có Nhiều Cầu Thủ Nhất', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Số Lượng')
axes[1,1].set_ylabel('Quốc Gia')

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

## 3. Tiền Xử Lý Dữ Liệu

In [ ]:
# Import preprocessing module
import sys
sys.path.append('..')
from preprocessing.data_preprocessing import FootballerDataPreprocessor

# Khởi tạo preprocessor
preprocessor = FootballerDataPreprocessor('../data/raw/sofifa_players.csv')
print("✅ Đã khởi tạo preprocessor!")

In [ ]:
# Thực hiện tiền xử lý
preprocessor.load_data() \
            .remove_unnecessary_columns() \
            .handle_missing_values(strategy='median') \
            .encode_categorical_features() \
            .handle_outliers(method='iqr', threshold=1.5) \
            .feature_engineering()

In [ ]:
# Chuẩn hóa dữ liệu
preprocessor.scale_features(scaler_type='standard')

In [ ]:
# Lấy dữ liệu đã xử lý
df_processed = preprocessor.get_processed_data()
print(f"\n✅ Dữ liệu đã xử lý: {df_processed.shape}")
df_processed.head()

In [ ]:
# Tổng quan dữ liệu sau xử lý
preprocessor.get_summary()

## 4. Lưu Dữ Liệu Đã Xử Lý

In [ ]:
# Lưu dữ liệu
output_path = '../data/processed/sofifa_players_processed.csv'
preprocessor.save_processed_data(output_path)

print("\n" + "=" * 80)
print("✅✅✅ HOÀN THÀNH TIỀN XỬ LÝ DỮ LIỆU! ✅✅✅")
print("=" * 80)

## 5. Tổng Kết

### Các bước tiền xử lý đã thực hiện:

1. **Loại bỏ cột không cần thiết**: Name, Player_URL, Team, Value_Raw, Wage_Raw, Unnamed columns

2. **Xử lý giá trị thiếu**: 
   - Numerical: Điền bằng median
   - Categorical: Điền bằng mode

3. **Mã hóa categorical**: 
   - Label Encoding cho: Nationality, Preferred_Foot, positioning, work_rate

4. **Xử lý outliers**: 
   - Sử dụng IQR method với threshold 1.5

5. **Feature Engineering**:
   - BMI (Body Mass Index)
   - Age_Group (Nhóm tuổi)
   - Potential_Diff (Tiềm năng phát triển)
   - Skill_Avg (Trung bình kỹ năng)
   - Physical_Score (Điểm thể lực)

6. **Chuẩn hóa dữ liệu**: 
   - StandardScaler (mean=0, std=1)

### Kết quả:
- Dữ liệu sạch, không có giá trị thiếu
- Tất cả features đã được chuẩn hóa
- Sẵn sàng cho modeling!